# Iterators and Generators   <a href="https://colab.research.google.com/github/Ahmad-Zaki/Python-Notes/blob/main/Iterators%20and%20Generators/iterators-and-generators.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open and Execute in Google Colaboratory"></a>

## Introduction

Iterators are objects that have you can loop\iterate upon, which means that we can go over all the values that it contains.

In Python, An iterable object is an object that implements `__iter__`, which is expected to return an iterator object. An iterator object implements `__next__`, which is expected to return the next element of the iterable object that returned it, and to raise a StopIteration exception when no more elements are available.

Python generators are a simple way of creating iterators. Generator functions allow you to declare a function that behaves like an iterator, i.e. it can be used in a for loop. A generator is, simply put, a function which can stop whatever it is doing at an arbitrary point in its body, return a value back to the caller, and, later on, resume from the point it had `frozen' and merrily proceed as if nothing had happened. 

## Iterators

Python contains a lot of iterable objects by default, like *lists*, *tuples*, *dictionaries*, ...etc. We can create iterators from them simply by passing them to the built-in function `iter()`:

In [1]:
lst = [1, 2, 3]
lst_iterator = iter(lst)
type(lst_iterator)

list_iterator

Once we have an itertor object, we can traverse the values of its iterable object one-by-one by calling the `next()` function :

In [2]:
print(next(lst_iterator))
print(next(lst_iterator))
print(next(lst_iterator))
print(next(lst_iterator))

1
2
3


StopIteration: 

Notice that when an iterator runs out of values to traverse, it raises `StopIteration` exception. This is how a for-loop knows when to stop the loop over iterators; once `StopIteration` is raised, the loop is over. 

With this logic, we can mimic what happens in a for loop as follows:

In [3]:
lst = [1, 2, 3]
lst_iterator = iter(lst)

In [4]:
print("for loop:")
for val in lst:
    print(val)

for loop:
1
2
3


In [5]:
print("equivalent while loop:")
while True:
    try:
        val = next(lst_iterator)
    except StopIteration:
        break
    
    print(val)

equivalent while loop:
1
2
3


You Can build your own iterables and iterators in Python as well. The `iter()` function calls the `__iter__` method of the object passed to it, and the `next()` function calls the `__next__` method. Therefore, you have to implement those methods in your class in order to make iterable/iterator objects of that class.

In [6]:
import random

class RandomIterable:
    def __iter__(self):
        return self

    def __next__(self):
        if random.choice(["go", "go", "stop"]) == "stop":
            print("iteration stopped!")
            raise StopIteration  # signals "the end"

        print("iteration is still going.")
        return 1

rand_iter = RandomIterable()

for val in rand_iter:
    pass

iteration is still going.
iteration stopped!


Notice that `__iter__` method returns `self`, this is because the object has a `__next__` method as well,which makes the object both an iterable and an iterator.

But we can make an iterable object only by implementing `__iter__` only, just remember that `__iter__` has to return an iterator object so that it can work properly.

In [7]:
class iterableObject:
    def __init__(self, vals: list):
        self.vals = vals

    def __iter__(self):
        return iter(self.vals)

In [8]:
a = iterableObject([1,2,3,4,5])

for v in a:
    print(v)

1
2
3
4
5


Iterators will typically need to maintain some kind of position state information (e.g., the index of the last element returned). If the iterable maintained that state itself, it would become inherently non-reentrant (meaning we could use it only one loop at a time). That's why it is not preferred to make an object an iterator of itself.

**In summary:** An object isn't iterable unless it provides `__iter__`, which returns an iterator. And for an object to be a valid iterator, it must provide `__next__`, which returns the next value in line, and raises `StopIteration` when no values are left.

## Generators

There is a bit of work involved with building an iterator in Python. We have to implement a class with `__iter__` and `__next__` methods, keep track of internal states, and raise StopIteration when there are no values to be returned.

Python generators are a simple way of creating iterators. Generator functions allow you to declare a function that behaves like an iterator, i.e. it can be used in a for loop.

Creating a generator is as simple as creating a function in Python, The main difference is that a generator will have a `yield` keyword instead of the usual `return` keyword.

### Example

In [9]:
def simple_generator():
	print('hello')
	yield 1
	print('world')
	yield 2

gen = simple_generator()
print(gen)

<generator object simple_generator at 0x000001CFF5551200>


Notice that calling the function did not result in the function getting executed. Instead, the Python interpreter gave us a `generator object`. This is because generators are *lazily evaluated*, meaning they will generate (i.e. yield) only one value at a time.

If we use `next()` on gen, it will execute the function, which generates one value and immediately stops. calling `next()` again will resume the execution of gen, generate another value and then stops again. Calling `next()` once there are no more values to yield will raise a `StopIteration` exception.

In [10]:
next(gen)

hello


1

In [11]:
next(gen)

world


2

This behaviour saves a lot of memory during runtime, especially when there are a large number of values to generate\go over.

we can see the difference with an example: theses two functions do the same task, but differ in how they do it. The first generates one value at a time, while the second computes values and returns them all at once in a list.

In [12]:
def first_n_gen(n: int):
    num = 0
    while num < n:
        yield num
        num += 1

def first_n_list(n: int):
    num = 0
    nums = []
    while num < n:
        nums.append(num)
        num += 1
    return nums

In [15]:
%timeit -r7 -n10 sum(first_n_gen(1_000_000))
%memit sum(first_n_gen(1_000_000))

151 ms ± 6.33 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
peak memory: 91.18 MiB, increment: 0.07 MiB


In [16]:
%timeit -r7 -n10 sum(first_n_list(1_000_000))
%memit sum(first_n_list(1_000_000))

197 ms ± 2.99 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
peak memory: 126.40 MiB, increment: 35.36 MiB


Notice that not only the list-based function is slower due to the overhead of list appends, but it also consumed significantly more memory as well. We could probably use list comprehension to bring the running time down, but the memory usage will remain far beyond what the generator used.

It may seem at this point that we should use generators everywhere in place of creating lists, but that would create
many complications.

What if, for example, you needed to reference the list of *first n numbers* multiple times? In this case, the list-based function would compute this list once and we won't need to do it again, while the generator would have to recompute them over and over again. 

### Built-in Generators

Many of Python’s built-in functions that operate on sequences are generators themselves (albeit sometimes a special type of generators). For example, `range` returns a generator of values as opposed to the actual list of numbers within the specified range. Similarly,
`map`, `zip`, `filter`, `reversed`, and `enumerate` all perform the calculation as needed and don’t store the full result. This means that the
operation `zip(range(100_000), range(100_000))` will always have only two numbers in memory in order to return its corresponding values, instead of precalculating the result for the entire range beforehand.

Consider the following generator, which generates the first n numbers of a febonacci series:

In [17]:
def feb_gen(n: int):
    a, b = 0, 1
    num = 0
    while num != n:
        yield a
        a, b = b, a+b
        num += 1

In [18]:
[*feb_gen(10)]

[0, 1, 1, 2, 3, 5, 8, 13, 21, 34]

for example, If we want to know how many numbers in the first 100,000 numbers of fibonacci sequence is divisible by 3, we can use list comprehension to create a list with these numbers and see its length:

In [19]:
len([i for i in feb_gen(100_000) if i % 3 == 0])

25000

While we are still using the generator to get these numbers, we created a list along the way to store them and get their count, which uses a lot of memory for no reason.

In [20]:
%memit len([i for i in feb_gen(100_000) if i % 3 == 0])

peak memory: 203.23 MiB, increment: 111.16 MiB


Instead of using a list comprehension statement and wasting all this memory, Python allows us to create **generator comprehension** statements! Theses statements are exactly like list comprehensions, but creates a generator that lazily yields each item one-by one.

In [21]:
filtered_gen = (i for i in feb_gen(100_000) if i % 3 == 0)
type(filtered_gen)

generator

Unfortunately, generators don't have a `length` property. Therefore, we have to work around that to get the count of these values:

In [22]:
sum(1 for _ in filtered_gen)

25000

In [23]:
%memit sum(1 for _ in filtered_gen)

peak memory: 92.69 MiB, increment: 0.04 MiB


As we can see, the memory usage of this generator is trivial compared to what we required with a list comprehension.

### Infinite Sequence Generators

Instead of calculating a known number of Fibonacci numbers, what if we instead attempted to calculate all of them?

In [24]:
def febonacci():
    a, b = 0, 1
    while True:
        yield a
        a, b = b, a+b

This generator does something that is impossible to do with functions that compute the whole sequence first and returns a list. It allows us to get as many numbers as we like from the febunacci sequence and stop when we determine that we had enough.

### Lazy Generator Evaluation

The way we get the memory benefits with a generator is by dealing only with the current values of interest. At any point in our calculation with a
generator, we have only the current value and cannot reference any other items in the sequence (algorithms that perform this way are generally called single pass or online). This can sometimes make generators more difficult to use, but many modules and functions can help.

The main library of interest is **`itertools`**, in the standard library. It supplies many useful functions, including these:
- `islice`: Allows slicing a potentially infinite generator
- `chain`: Chains together multiple generators
- `takewhile`: Adds a condition that will end a generator
- `cycle`: Makes a finite generator infinite by constantly repeating it